# Instalaciones y librerías necesarias

In [ ]:
!apt-get install ffmpeg
!pip install -q mediapy

In [25]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import subprocess
import mediapy as media

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Motion detection

In [40]:
def process_frame_difference(new_image, prev_image, movement=False, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)

    #cv2_imshow(thresh)

    # Encontrar componentes conectadas
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(thresh)

    # Filtrar componentes por área
    min_area = 1000
    filtered_mask = np.zeros_like(prev_image)
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]

        # Calcular el aspect ratio del rectángulo
        aspect_ratio = w / float(h)

        # Filtrar por área y aspect ratio
        if area > min_area:
            # Acumular la región en la máscara
            filtered_mask[labels == i] = 255
            # Establecer movimiento en la imagen
            movement = True

    #cv2_imshow(filtered_mask)

    return movement

In [ ]:
# Ruta del video
video_path = '/content/drive/My Drive/computer_vision/video_emi.mp4'
video_path_reduced = '/content/drive/My Drive/computer_vision/video_emi_reduced.mp4'

# Comando para reducir la resolución del video original
ffmpeg_command = f'ffmpeg -i "{video_path}" -vf "scale=-1:180" "{video_path_reduced}"'

# Ejecutar el comando de ffmpeg para reducir la resolución
subprocess.call(ffmpeg_command, shell=True)

# Cargar el video
cap = cv2.VideoCapture(video_path_reduced)

# Leer el primer fotograma
ret, prev_frame = cap.read()

# Bucle para procesar cada fotograma
while cap.isOpened():
    # Leer el fotograma actual
    ret, new_frame = cap.read()
    if not ret:
        break  # Salir del bucle si no hay más fotogramas

    # Procesar el par de fotogramas
    processed_frame = process_frame_difference(new_frame, prev_frame)
    if processed_frame == True:
        print("Movimiento detectado")

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Actualizar el fotograma anterior
    prev_frame = new_frame


# Liberar recursos y cerrar ventanas
cap.release()
cv2.destroyAllWindows()

# Flujo óptico denso

In [31]:
# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas
            prev_image = None  # Inicializar la imagen previa

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # Actualizar la imagen previa
                prev_image = new_image.copy()

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

In [32]:
# Función para procesar el flujo óptico denso
def process_dense_optical_flow(new_image, prev_image):
    # Convierte la nueva imagen a escala de grises
    gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    if not hasattr(process_dense_optical_flow, "init_done"):
        process_dense_optical_flow.prev_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        process_dense_optical_flow.mask = np.zeros_like(new_image)
        process_dense_optical_flow.mask[..., 1] = 255
        process_dense_optical_flow.init_done = True

    if process_dense_optical_flow.init_done:
        prev_gray = process_dense_optical_flow.prev_gray
        mask = process_dense_optical_flow.mask

    # Calcula el flujo óptico
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Computa magnitud y ángulo de los vectores 2D
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Establece el tono de la imagen según la dirección del flujo óptico
    mask[..., 0] = angle * 180 / np.pi / 2
    # Establece el valor de la imagen según la magnitud del flujo óptico
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convierte de HSV a RGB
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    # Actualiza la imagen previa a gris
    process_dense_optical_flow.prev_grayprev_gray = gray.copy()
    return rgb

In [ ]:
# Ruta del video
video_path = '/content/drive/My Drive/computer_vision/video_emi.mp4'
video_path_reduced = '/content/drive/My Drive/computer_vision/video_emi_reduced.mp4'

# Comando para reducir la resolución del video original
ffmpeg_command = f'ffmpeg -i "{video_path}" -vf "scale=-1:180" "{video_path_reduced}"'

# Ejecutar el comando de ffmpeg para reducir la resolución
subprocess.call(ffmpeg_command, shell=True)

# Nombres de los archivos de video de entrada y salida
filename_in = video_path_reduced
filename_out = '/content/drive/My Drive/computer_vision/video_emi_dense.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_dense_optical_flow,
                max_time=20)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

# Flujo óptico disperso

In [38]:
def process_sparse_optical_flow(new_image, prev_image):
    # Preparamos las imagenes de trabajo
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    prev_gray_image = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)

    # Verificar si ya se han detectado las características de Shi-Tomasi
    if not hasattr(process_sparse_optical_flow, "shi_tomasi_done"):
        # Definir parámetros para la detección de esquinas de Shi-Tomasi
        feature_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)
        # Detectar puntos característicos en la imagen
        process_sparse_optical_flow.prev_points = cv2.goodFeaturesToTrack(new_gray, mask=None, **feature_params)
        # Crear una máscara para dibujar el flujo óptico
        process_sparse_optical_flow.mask = np.zeros_like(new_image)
        # Marcar que se ha completado la detección de Shi-Tomasi
        process_sparse_optical_flow.shi_tomasi_done = True

    # Continuar si se ha completado la detección de Shi-Tomasi
    if process_sparse_optical_flow.shi_tomasi_done:
        prev_points = process_sparse_optical_flow.prev_points
        mask = process_sparse_optical_flow.mask

    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calcular el flujo óptico de Lucas-Kanade
    new_points, status, error = cv2.calcOpticalFlowPyrLK(prev_gray_image, new_gray, prev_points, None, **lk_params)
    # Filtrar puntos buenos
    good_old = prev_points[status == 1]
    good_new = new_points[status == 1]
    color = (0, 255, 0)  # Color para el dibujo
    # Dibujar el movimiento (flujo óptico)
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.astype(int).ravel()
        c, d = old.astype(int).ravel()
        mask = cv2.line(mask, (a, b), (c, d), color, 2)
        new_image = cv2.circle(new_image, (a, b), 3, color, -1)

    # Combinar la imagen actual con las líneas de flujo óptico dibujadas
    output = cv2.add(new_image, mask)
    # Actualizar puntos para el siguiente cuadro
    process_sparse_optical_flow.prev_points = good_new.reshape(-1, 1, 2)

    return output

In [ ]:
# Ruta del video
video_path = '/content/drive/My Drive/computer_vision/video_emi.mp4'
video_path_reduced = '/content/drive/My Drive/computer_vision/video_emi_reduced.mp4'

# Comando para reducir la resolución del video original
ffmpeg_command = f'ffmpeg -i "{video_path}" -vf "scale=-1:180" "{video_path_reduced}"'

# Ejecutar el comando de ffmpeg para reducir la resolución
subprocess.call(ffmpeg_command, shell=True)

# Nombres de los archivos de video de entrada y salida
filename_in = video_path_reduced
filename_out = '/content/drive/My Drive/computer_vision/video_emi_sparse.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_sparse_optical_flow,
                max_time=20)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

# Reflexión de resultados

En cuanto al primer procesamiento de detección de movimiento, se encontraron resultados favorables y que suponen de utilidad para aplicaciones de seguridad. Esto se logró realizando una umbralización de la imagen, para luego encontrar los componentes conectados y filtrarlos por area mínima (adicionalmente se podría establecer un filtro de relación de aspecto).

Luego, en ambos procesamientos de flujo óptico, se realizan utilizando el código visto en clases. Los resultados obtenidos no son del todo aceptables ya que la detección de movimiento se genera desde el inico del video (por cambios normales en cualquier video), afectando al objetivo de las técnicas de hacer seguimiento a objetos.
Una solución a esto podría ser preprocesar las imágenes con técnicas de morfología.